# Examples of market data queries


## Query Name: Most recent price of BTC and ETH

```sql
/* Find the latest value available for each symbol (crypto pair).
 This dataset collects live market data from the OKX API. */
SELECT * FROM trades
WHERE symbol in ('BTC-USDT', 'ETH-USDT')
LATEST ON timestamp PARTITION BY symbol;
```

---

## Query Name: Bar chart (OHLC) downsampled with a 15 minutes interval

```sql
/* Aggregations for the BTC-USDT for today downsampled in 15-minute intervals.
 We use the SQL extension SAMPLE BY to aggregate data at regular intervals. QuestDB
 ingests live market data from the OKX API. */
SELECT
    timestamp, symbol,
    first(price) AS open,
    last(price) AS close,
    min(price),
    max(price),
    sum(amount) AS volume
FROM trades
WHERE symbol = 'BTC-USDT' AND timestamp IN today()
SAMPLE BY 15m;
```
---


## Query Name: VWAP Bitcoin price, cumulative with a 10 minute interval for one day

```sql
/* Calculates the weighted average for BTC-USDT for yesterday, cumulative in 10-minute intervals.
 We use the SQL extension SAMPLE BY to aggregate data at regular intervals.
 Then we use window functions to calculate cumulative values and get the VWAP
QuestDB ingests live market data from the OKX API */
WITH btc_usdt AS (
    SELECT
          timestamp,  symbol,
          SUM(amount) AS volume,
          SUM(price * amount) AS traded_value
     FROM trades
     WHERE timestamp IN today()
     AND symbol = 'BTC-USDT'
     SAMPLE BY 10m
), cumulative AS (
     SELECT timestamp, symbol,
           SUM(traded_value)
                OVER (ORDER BY timestamp) AS cumulative_value,
           SUM(volume)
                OVER (ORDER BY timestamp) AS cumulative_volume
     FROM btc_usdt
)
SELECT *, cumulative_value/cumulative_volume AS vwap FROM cumulative;
```
---

## Query Name: Hourly returns (%) of BTC

```sql
/* Calculates the hourly returns in % for BTC-USDT for the last 30 days.
 We use SAMPLE BY to aggregate the data at regular intervals - in this case daily.
 The previous value is obtained with the first_value() window function */
select timestamp, round(100*(last_price - prev_price) / prev_price, 2) as return_pct
from
(
  select timestamp, last_price, first_value(last_price) over (rows between 1 preceding and 1 preceding) as prev_price
  from
  (
    SELECT timestamp, last(price) last_price
    FROM trades
    WHERE symbol = 'BTC-USDT'
    and timestamp > dateadd('d', -30, now())
    SAMPLE BY 1h
  )
)
where prev_price is not null;
```
---

## Query Name: Find gaps in your trading data

```sql
/* Aggregates data in 1 minute intervals, interpolating with empty values any gaps where we do not
 have sell operations for each tracked symbol. We use the SAMPLE BY keyword to aggregate data per minute,
using the FILL(NULL) keyword to instruct QuestDB to output an empty column for any gaps. We could have also used
 FILL(LINEAR) or FILL(PREV) for different interpolation strategies.
 Note we are using DECLARE to define the SAMPLE interval and the time_range. You could try
 values such as @sample_interval := 30s, @time_range := yesterday(),
 or @time_range := INTERVAL( interval_start( yesterday ()), now() ) */
DECLARE
  @sample_interval := 1m,
  @time_range := today()
SELECT
   timestamp, symbol, side, sum(amount) as volume
 FROM trades
 WHERE side = 'sell' AND timestamp IN @time_range
 SAMPLE BY @sample_interval FILL(NULL);
```
---

## Query Name: 30d and 15d Moving Average

```sql
/* Calculates the rolling moving average of BTC-USDT using Window Functions */
SELECT timestamp time, symbol, price as priceBtc,
       avg(price) over (PARTITION BY symbol ORDER BY timestamp RANGE between 15 minutes PRECEDING AND CURRENT ROW) moving_avg_15_minutes,
       avg(price) over (PARTITION BY symbol ORDER BY timestamp RANGE between 30 minutes PRECEDING AND CURRENT ROW) moving_avg_30_minutes
FROM trades
WHERE timestamp > dateadd('M', -1, now())
AND symbol = 'BTC-USDT';
```
---

## Query Name: Implied BTC-ETH exchange rate

```sql
/* This query calculates the implied BTC-ETH exchange rate using BTC-USDT and
 ETH-USDT quotes from the OKX API.
 We use ASOF JOIN, a fuzzy timestamp based lookup from one table to another. */
WITH btc AS (
    SELECT timestamp, price
    FROM trades
    WHERE symbol = 'BTC-USDT' AND timestamp > dateadd('d', -30, now())
),
eth AS (
    SELECT timestamp, price
    FROM trades
    WHERE symbol = 'ETH-USDT' and timestamp > dateadd('d', -30, now())
)
SELECT
    btc.timestamp btc_time,
    btc.price btc_price,
    eth.price eth_price,
    round(btc.price/eth.price, 3) btc_to_eth_ratio
FROM btc
ASOF JOIN eth;
```
---

## Query Name: Most recent sell event for each buy

```sql
/* This query finds each buy event, and joins with the most
 recent sell event for the same symbol.
 */
WITH trade_buys AS  (
 SELECT timestamp, symbol, price, side
 FROM trades
 WHERE side = 'buy' 
 ), trade_sells AS  (
         SELECT timestamp, symbol, price, side
 FROM trades
 WHERE side = 'sell' 
 )
 SELECT * from trade_buys ASOF JOIN trade_sells ON (symbol);
```
---


## Query Name: Significant prices only

```sql
/* Select only rows with different price than previous row for same symbol and side.
 We use the LAG function to get the value from the previous row for the same symbol and side */
WITH trade_and_previous AS (
SELECT timestamp, symbol, side, price,
       LAG(price) OVER(PARTITION BY symbol, side ORDER BY timestamp) as prev_price
FROM trades
WHERE timestamp IN today()
)
select * from trade_and_previous where price <> prev_price;
```
---


## Query Name: The weighted mid price for bids and asks

```sql
/* The weighted mid price for bids and asks.
We first separate our trades table into bids and
asks depending on side, and then we aggregate at
1 second intervals. Then we use ASOF JOIN to
get the closest ask for the bid, and use the
wmid function  */
WITH bids AS (
SELECT timestamp AS bid_ts, symbol,
   amount AS bid_size,
   avg(price*amount) AS bid_price
 FROM trades WHERE side = 'buy'  
 SAMPLE BY 1s
), asks AS  (
 SELECT timestamp AS ask_ts, symbol,
   amount AS ask_size,
   avg(price*amount) AS ask_price
 FROM trades  WHERE side = 'sell'
 SAMPLE BY 1s
)
SELECT bid_ts, bids.symbol, bid_size, bid_price,
wmid(bid_price, bid_size, ask_price, ask_size) AS weighted_mid_price,
ask_price, ask_size,
 FROM bids ASOF JOIN asks ON (symbol) where ask_price is not null;
```
---

## Query Name: Query trade data from a parquet file

```sql
/*
This query reads from a sample parquet file that is now packaged with QuestDB as of
version 8.1.1. It demonstrates how one can seamlessly integrate data in open
formats with the QuestDB query engine.
*/
select * from read_parquet('trades.parquet');
```
---

## Query Name: Query trade data from parquet using time-series extensions

```sql
/*
This query reads from a sample parquet file that is now packaged with QuestDB as of
version 8.1.1. It demonstrates how one can assign the designated timestamp
when querying over an external table, so functions like SAMPLE BY can be used*/
select timestamp, avg(price) from (read_parquet('trades.parquet') timestamp(timestamp)) sample by 15m;
```
---


## Query Name: Find price trend boundaries

```sql
/* Find trades where an upwards or downwards trend changed direction.
   We use LAG to get the previous value for each row, then we use
   CASE to output 1, 0, or NULL depending on the comparison with
   the previous row, then we use LAG with IGNORE NULLS, so we
   are ignoring all the values that had no changes in the trend,
   and we can finally show only the rows where there was a trend
   change, together with the difference from the previous price.
*/
WITH trade_and_previous AS (
SELECT timestamp, symbol, price,
       LAG(price) OVER(PARTITION BY symbol ORDER BY timestamp) as prev_price
FROM trades
WHERE timestamp > dateadd('w', -1, now())
AND symbol = 'BTC-USDT' AND side = 'buy'
),
trends AS (
SELECT *,
  CASE WHEN price > prev_price THEN 1
       WHEN price < prev_price THEN 0 ELSE NULL END AS trend
FROM trade_and_previous
), prev_trends AS (
SELECT *,
      LAG(trend) IGNORE NULLS OVER(PARTITION BY SYMBOL ORDER BY TIMESTAMP) as prev_trend
FROM trends
)
SELECT timestamp, symbol, price, prev_price, price - prev_price as price_delta
from prev_trends where trend is not null and trend <> prev_trend;
```
---

